# 기본 전처리 및 단어 사전 생성 코드
### 코더: 박연우
### ver.03 : 바른 형태소 분석 API

In [2]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
# import wandb
# from wandb.keras import WandbCallback
from sklearn.preprocessing import LabelEncoder
import re
import numpy as np

In [3]:
# train_data_path ="~/aiffel/dktc/data/train.csv"

train_data_path = "/content/drive/MyDrive/aiffel/dktc/data/train.csv"
train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3950 entries, 0 to 3949
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idx           3950 non-null   int64 
 1   class         3950 non-null   object
 2   conversation  3950 non-null   object
dtypes: int64(1), object(2)
memory usage: 92.7+ KB


In [5]:
CLASS_NAMES = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화']

encoder = LabelEncoder()
encoder.fit(CLASS_NAMES)

train_data['class'] = encoder.transform(train_data['class'])

corpus = train_data["conversation"]

In [6]:
corpus.head()

,conversation
0,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


### 데이터 클래스별로 분리

In [50]:
print(train_data['class'])

0       3
1       3
2       1
3       0
4       0
       ..
3945    1
3946    0
3947    2
3948    0
3949    2
Name: class, Length: 3950, dtype: int64


In [93]:
def train_class_split(train_data) :  # data를 class 별로 분리
  train_classes = [pd.DataFrame() for _ in range(4)]

  for i in range(4) :
    train_classes[i] = train_data[train_data['class'] == i]

  return train_classes

In [94]:
train_classes = train_class_split(train_data)

In [95]:
train_classes[0].head()

,idx,class,conversation
3,3,0,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,0,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
9,9,0,자네 세금계산서 보내준 거 메일이랑 수치가 다르던데?\n그게.\n얼마나 해먹었어? ...
10,10,0,네 뭐 드릴까요?\n말보르 레드 하나.\n네. 여기요!\n여기 사장은 없어?\n네 ...
13,13,0,형! 형! 돈좀있어?\n저 저요.?\n여기 형말고 누가 있어? 형 얼마 있어?\n저...


In [96]:
train_classes[1].head()

,idx,class,conversation
2,2,1,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
6,6,1,35번 손님 아이스커피 두잔나왔습니다\n아이스커피? \n네 맛있게드세요\n저기요 아...
7,7,1,경비아저씨 내차에 경고장 붙였어요?\n내가 여기 몇년째 사는데 아직도 내차인줄 모르...
8,8,1,이거 할인 된다면서요\n 확인해보겠습니다\n 아까 저 시람은 할인 해주던데\n 네 ...
18,18,1,야 너이마빡에 빛이난다\n어?\n야 한대만 때려보자 퍽\n아 흑 왜그래 \n아팠어?...


In [97]:
train_classes[2].head()


,idx,class,conversation
5,5,2,나 이틀뒤에 가나다 음식점 예약좀 해줘. 저녁7시로.\n가나다 음식점이요.?\n응....
11,11,2,이대리 오늘 도대체 뭘한거야?\n창고정리도 하고 바닥도 청소했습니다\n이게 한거니?...
14,14,2,김사원\n대리님 왜 그러세요?\n이거 오늘까지 제출해야 하는데 좀 해줄래?\n네? ...
19,19,2,나씨 정수기 물탱크 좀 갈아주시겠어요?\n싫습니다. 물 저만 마시는 것도 아닌데 매...
36,36,2,나이가 몇인데 내가 이런 것까지 일러줘야돼?\n죄송합니다\n기본적인 것도 탑재가 안...


In [98]:
train_classes[3].head()

,idx,class,conversation
0,0,3,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,3,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
12,12,3,네네 무슨 일 때문에 전화주셨나요?\n우리 애가 지우개 하나 훔친거 가지고 애들 앞...
20,20,3,너가 민수라는 남자 제자니?\n네 맞는데요 왜 그러시죠? 혹시 저희 선생님께 무슨 ...
23,23,3,나좀 제발 그만 따라다녀.부탁이야\n너 만나려면 널 찾아가야될거아냐\n우리 헤어졌자...


In [99]:
# train_classes[0]에서 conversation 열만 추출
conversations_class_0 = train_classes[0]['conversation']
conversations_class_0.head()

,conversation
3,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
9,자네 세금계산서 보내준 거 메일이랑 수치가 다르던데?\n그게.\n얼마나 해먹었어? ...
10,네 뭐 드릴까요?\n말보르 레드 하나.\n네. 여기요!\n여기 사장은 없어?\n네 ...
13,형! 형! 돈좀있어?\n저 저요.?\n여기 형말고 누가 있어? 형 얼마 있어?\n저...


In [100]:
# train_classes[1]에서 conversation 열만 추출
conversations_class_1 = train_classes[1]['conversation']
conversations_class_1.head()

,conversation
2,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
6,35번 손님 아이스커피 두잔나왔습니다\n아이스커피? \n네 맛있게드세요\n저기요 아...
7,경비아저씨 내차에 경고장 붙였어요?\n내가 여기 몇년째 사는데 아직도 내차인줄 모르...
8,이거 할인 된다면서요\n 확인해보겠습니다\n 아까 저 시람은 할인 해주던데\n 네 ...
18,야 너이마빡에 빛이난다\n어?\n야 한대만 때려보자 퍽\n아 흑 왜그래 \n아팠어?...


In [101]:
# train_classes[2]에서 conversation 열만 추출
conversations_class_2 = train_classes[2]['conversation']
conversations_class_2.head()

,conversation
5,나 이틀뒤에 가나다 음식점 예약좀 해줘. 저녁7시로.\n가나다 음식점이요.?\n응....
11,이대리 오늘 도대체 뭘한거야?\n창고정리도 하고 바닥도 청소했습니다\n이게 한거니?...
14,김사원\n대리님 왜 그러세요?\n이거 오늘까지 제출해야 하는데 좀 해줄래?\n네? ...
19,나씨 정수기 물탱크 좀 갈아주시겠어요?\n싫습니다. 물 저만 마시는 것도 아닌데 매...
36,나이가 몇인데 내가 이런 것까지 일러줘야돼?\n죄송합니다\n기본적인 것도 탑재가 안...


In [102]:
# train_classes[3]에서 conversation 열만 추출
conversations_class_3 = train_classes[3]['conversation']
conversations_class_3.head()

,conversation
0,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
12,네네 무슨 일 때문에 전화주셨나요?\n우리 애가 지우개 하나 훔친거 가지고 애들 앞...
20,너가 민수라는 남자 제자니?\n네 맞는데요 왜 그러시죠? 혹시 저희 선생님께 무슨 ...
23,나좀 제발 그만 따라다녀.부탁이야\n너 만나려면 널 찾아가야될거아냐\n우리 헤어졌자...


In [103]:
# 각 DataFrame을 CSV 파일로 저장
for i, df in enumerate(train_classes):
    df.to_csv(f'train_class_{i}.csv', index=False)

# 각 CSV 파일에 대한 다운로드 링크 제공 (Jupyter 환경에서 실행 시)
from IPython.display import FileLink, display

for i in range(len(train_classes)):
    display(FileLink(f'train_class_{i}.csv')) # 링크 에러시, 코랩 폴더 열면, 파일이 생성되어 있을 것임


/content/train_class_0.csv

/content/train_class_1.csv

/content/train_class_2.csv

/content/train_class_3.csv

### 전처리

In [7]:
# 전처리 함수 수정 ver.

def preprocess_sentence(sentence):
    sentence = sentence.strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

    sentence = re.sub(r"[^가-힣a-zA-Z?.!,]+", " ", sentence)

    sentence = sentence.strip()
    return sentence


In [8]:

def load_conversations_from_df(corpus_df):
    conv_data = []

    for index, row in corpus_df.iterrows():  # 수정된 부분
        conversations = row['conversation']  # 수정된 부분
        conv_data.append(preprocess_sentence(conversations))  # 전처리 함수 적용
    return conv_data

In [ ]:

conv_data = load_conversations_from_df(train_data)

print('전체 샘플 수:', len(conv_data))

전체 샘플 수: 3950


In [ ]:
# 한국어 전처리 함수 적용 결과

print('전처리 후의 1081번째 샘플: {}'.format(conv_data[1081]))
print('전처리 후의 500번째 샘플: {}'.format(conv_data[500]))


전처리 후의 1081번째 샘플: 아까 시에 배민에서 스테이크 덮밥 시켰던 사람인데요 시요 ? 벌써 시간이나 지났는데 . 무슨 일이신데요 ? 손님한테 그렇게 밖에 말을 못해요 ? 아닙니다 죄송합니다 오늘 스테이크에 제대로 된 고기 쓰시는 것 맞아요 ? 네 저희는 제대로 수입된 고기 납품받아 조리하고 있습니다 . 고기가 너무 질겨서 아이가 씹지도 못했어요 . 이거 환불해주세요 . 손님 배달 특성상 조리 후에 바로 배달된다해도 고기의 육질상태가 시간이 흐름에 따라 저하되는 것은 당연합니다 . 지금 말 다했어요 ? 그리고 반 이상 드셔놓고 환불해달라고 하시면 어떡해요 . 그럼 지금 고객인 내가 잘못했다는 거에요 ? 이거 안되겠네 . 당장 업체 본사에 전화하고 배민 고객센터에도 다 신고할거에요 . 당신 이 바닥에서 장사 못 해 이제 .
전처리 후의 500번째 샘플: 자기야 나 필요한거 있는뎅 . ! 어떤거 ? 음 나 가방 필요해서 그런데 만원만 주랑 웅 . ? 갑자기 ? 자기 며칠 전에도 가방 사줬잖아 그거는 내 생일선물이였구우 이번꺼는 그냥 여자친구한테 선물 줄 수 있는거잖아앙 ! 음자기가 이런식으로 매년 나한테 요구하는 돈이 얼만지 알아 ? 년마다 단위인데 우리 미래 생각하려면 돈 아껴야지 . ! 헐 모야잉 자기야 어차피 결혼하면 자기 돈 내 돈 상관 없이 같이 쓸텐데 나한테 쓰는게 아까워 ? ! 가방 살 돈 안주면 화낸다 ? ! 그게 아니라 그럼 사죠 돈 보내놔 알겠징 ? ! 자기야 사랑행 하 그래


# 버전 03

In [9]:
# 바른의 리눅스 설치본 다운로드

!curl -LJks -H "uname:$(uname -a)" https://bareun.ai/api/get -o bareun-linux.deb
!ls

bareun-linux.deb  drive  sample_data


In [10]:
# 바른 설치

!uname -a
!dpkg -i bareun-linux.deb

Linux 5eb84f61b40b 6.1.85+ #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024 x86_64 x86_64 x86_64 GNU/Linux
Selecting previously unselected package bareun.
(Reading database ... 123620 files and directories currently installed.)
Preparing to unpack bareun-linux.deb ...
Unpacking bareun (2.3.0) ...
Setting up bareun (2.3.0) ...
System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down
System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to connect to bus: Host is down
Configuration file /etc/systemd/system/bareun.service is marked executable. Please remove executable permission bits. Proceeding anyway.
Created symlink /etc/systemd/system/multi-user.target.wants/bareun.service → /etc/systemd/system/bareun.service.


In [11]:
# 바른 모델이 GPU를 사용하기 위해서 다운받는 tensorflow 라이브러리

!curl -O https://storage.googleapis.com/tensorflow/libtensorflow/libtensorflow-gpu-linux-x86_64-2.9.1.tar.gz
!tar -C /opt/bareun -xzf libtensorflow-gpu-linux-x86_64-2.9.1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  392M  100  392M    0     0  71.9M      0  0:00:05  0:00:05 --:--:-- 76.6M


In [12]:
%env BAREUN_ROOT="/opt/bareun"
%env LD_LIBRARY_PATH="/opt/bareun/lib"
!BAREUN_ROOT="/opt/bareun" LD_LIBRARY_PATH="/opt/bareun/lib" nohup /opt/bareun/bin/bareun&

env: BAREUN_ROOT="/opt/bareun"
env: LD_LIBRARY_PATH="/opt/bareun/lib"
nohup: appending output to 'nohup.out'


In [13]:
!ps -ef | grep bareun

root        1732       1  0 18:24 ?        00:00:00 /opt/bareun/bin/bareun
root        1737     316  0 18:24 ?        00:00:00 /bin/bash -c ps -ef | grep bareun
root        1739    1737  0 18:24 ?        00:00:00 grep bareun


In [14]:
# Yeonwoo's API KEY

!BAREUN_ROOT="/opt/bareun" LD_LIBRARY_PATH="/opt/bareun/lib" /opt/bareun/bin/bareun -reg koba-MDIHR4Q-BQUUMOY-TOGE42A-NDESPKY

2024-10-04 18:24:20.318898: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/bareun/lib
Use config file: /opt/bareun/config/bareun.json

------
200 OK
sucessfully registered koba-MDIHR4Q-BQUUMOY-TOGE42A-NDESPKY


In [15]:
!pip install -U bareunpy

In [16]:
import sys
import bareunpy as brn
import google.protobuf.text_format as tf

from bareunpy import Tagger   # 이하 추가
from bareunpy import Tokenizer

In [17]:
# 아래에 "https://bareun.ai/"에서 이메일 인증 후 발급받은 API KEY("koba-...")를 입력해주세요. "로그인-내정보 확인"

API_KEY = "koba-MDIHR4Q-BQUUMOY-TOGE42A-NDESPKY" # <- 본인의 API KEY로 교체
t = brn.Tagger(API_KEY, "localhost", 5656)
res = t.tags(["안녕하세요. 정말 좋은 날씨네요."])
m = res.msg()
tf.PrintMessage(m, out=sys.stdout, as_utf8=True)

sentences {
  text {
    content: "안녕하세요. 정말 좋은 날씨네요."
    length: 18
  }
  tokens {
    text {
      content: "안녕하세요."
      length: 6
    }
    morphemes {
      text {
        content: "안녕하"
        length: 3
      }
      tag: VA
      out_of_vocab: IN_BUILTIN_DICT
    }
    morphemes {
      text {
        content: "시"
        begin_offset: 3
        length: 1
      }
      tag: EP
      probability: 0.9998893
    }
    morphemes {
      text {
        content: "어요"
        begin_offset: 3
        length: 2
      }
      tag: EF
      probability: 0.9899567
    }
    morphemes {
      text {
        content: "."
        begin_offset: 5
        length: 1
      }
      tag: SF
      probability: 0.99979943
    }
    lemma: "안녕하"
    tagged: "안녕하/VA+시/EP+어요/EF+./SF"
  }
  tokens {
    text {
      content: "정말"
      begin_offset: 7
      length: 2
    }
    morphemes {
      text {
        content: "정말"
        begin_offset: 7
        length: 2
      }
      tag: MAG
      probabili

### API에서 분석 받은 객체의 정보를 확인

- length of tokens in sentences
- length of morphemes of first token in sentences
- lemma of first token in sentences
- first morph of first token in sentences
- tag of first morph of first token in sentences

In [16]:
print(f'length of tokens in sentences[0] is {len(m.sentences[0].tokens)}')
print(f'length of morphemes of first token in sentences[0] is {len(m.sentences[0].tokens[0].morphemes)}')
print(f'lemma of first token in sentences[0] is {m.sentences[0].tokens[0].lemma}')
print(f'first morph of first token in sentences[0] is {m.sentences[0].tokens[0].morphemes[0]}')
print(f'tag of first morph of first token in sentences[0] is {m.sentences[0].tokens[0].morphemes[0].tag}')

length of tokens in sentences[0] is 4
length of morphemes of first token in sentences[0] is 4
lemma of first token in sentences[0] is 안녕하
first morph of first token in sentences[0] is text {
  content: "\354\225\210\353\205\225\355\225\230"
  length: 3
}
tag: VA
out_of_vocab: IN_BUILTIN_DICT

tag of first morph of first token in sentences[0] is 38


### 형태소 분석 실행한 객체를 json 형식으로 변환

In [ ]:
# get json object
jo = res.as_json()
print(jo)

{'sentences': [{'text': {'content': '안녕하세요. 정말 좋은 날씨네요.', 'length': 18, 'beginOffset': 0}, 'tokens': [{'text': {'content': '안녕하세요.', 'length': 6, 'beginOffset': 0}, 'morphemes': [{'text': {'content': '안녕하', 'length': 3, 'beginOffset': 0}, 'tag': 'VA', 'outOfVocab': 'IN_BUILTIN_DICT', 'probability': 0.0}, {'text': {'content': '시', 'beginOffset': 3, 'length': 1}, 'tag': 'EP', 'probability': 0.9998893, 'outOfVocab': 'IN_WORD_EMBEDDING'}, {'text': {'content': '어요', 'beginOffset': 3, 'length': 2}, 'tag': 'EF', 'probability': 0.9899567, 'outOfVocab': 'IN_WORD_EMBEDDING'}, {'text': {'content': '.', 'beginOffset': 5, 'length': 1}, 'tag': 'SF', 'probability': 0.99979943, 'outOfVocab': 'IN_WORD_EMBEDDING'}], 'lemma': '안녕하', 'tagged': '안녕하/VA+시/EP+어요/EF+./SF', 'modified': ''}, {'text': {'content': '정말', 'beginOffset': 7, 'length': 2}, 'morphemes': [{'text': {'content': '정말', 'beginOffset': 7, 'length': 2}, 'tag': 'MAG', 'probability': 0.9906673, 'outOfVocab': 'IN_WORD_EMBEDDING'}], 'lemma': '정말',

### 형태소 분석 실행한 객체를 튜플로 저장하거나, 명사, 동사만 추출

In [17]:
# get tuple of pos tagging.
pa = res.pos()
print(pa)
# another methods
ma = res.morphs()
print(ma)
na = res.nouns()
print(na)
va = res.verbs()
print(va)



[('안녕하', 'VA'), ('시', 'EP'), ('어요', 'EF'), ('.', 'SF'), ('정말', 'MAG'), ('좋', 'VA'), ('은', 'ETM'), ('날씨', 'NNG'), ('이', 'VCP'), ('네', 'EF'), ('요', 'JX'), ('.', 'SF')]
['안녕하', '시', '어요', '.', '정말', '좋', '은', '날씨', '이', '네', '요', '.']
['날씨']
[]


### tagger

In [18]:
# 태깅 코드

tagger = brn.Tagger(API_KEY, "localhost", 5656)

# print results.
res = tagger.tags(["안녕하세요.", "반가워요!"])
print(res.msg()) # 확인용 코드

sentences {
  text {
    content: "\354\225\210\353\205\225\355\225\230\354\204\270\354\232\224."
    length: 6
  }
  tokens {
    text {
      content: "\354\225\210\353\205\225\355\225\230\354\204\270\354\232\224."
      length: 6
    }
    morphemes {
      text {
        content: "\354\225\210\353\205\225\355\225\230"
        length: 3
      }
      tag: VA
      out_of_vocab: IN_BUILTIN_DICT
    }
    morphemes {
      text {
        content: "\354\213\234"
        begin_offset: 3
        length: 1
      }
      tag: EP
      probability: 0.9999606013298035
    }
    morphemes {
      text {
        content: "\354\226\264\354\232\224"
        begin_offset: 3
        length: 2
      }
      tag: EF
      probability: 0.9929438233375549
    }
    morphemes {
      text {
        content: "."
        begin_offset: 5
        length: 1
      }
      tag: SF
      probability: 0.9998064041137695
    }
    lemma: "\354\225\210\353\205\225\355\225\230"
    tagged: "\354\225\210\353\205\22

In [19]:
# get protobuf message.
m = res.msg()
tf.PrintMessage(m, out=sys.stdout, as_utf8=True)
print(tf.MessageToString(m, as_utf8=True))
print(f'length of sentences is {len(m.sentences)}')

sentences {
  text {
    content: "안녕하세요."
    length: 6
  }
  tokens {
    text {
      content: "안녕하세요."
      length: 6
    }
    morphemes {
      text {
        content: "안녕하"
        length: 3
      }
      tag: VA
      out_of_vocab: IN_BUILTIN_DICT
    }
    morphemes {
      text {
        content: "시"
        begin_offset: 3
        length: 1
      }
      tag: EP
      probability: 0.9999606
    }
    morphemes {
      text {
        content: "어요"
        begin_offset: 3
        length: 2
      }
      tag: EF
      probability: 0.9929438
    }
    morphemes {
      text {
        content: "."
        begin_offset: 5
        length: 1
      }
      tag: SF
      probability: 0.9998064
    }
    lemma: "안녕하"
    tagged: "안녕하/VA+시/EP+어요/EF+./SF"
  }
  refined: "안녕하세요."
}
sentences {
  text {
    content: "반가워요!"
    begin_offset: 7
    length: 5
  }
  tokens {
    text {
      content: "반가워요!"
      begin_offset: 7
      length: 5
    }
    morphemes {
      text {
        con

In [20]:
## output : 2
print(f'length of tokens in sentences[0] is {len(m.sentences[0].tokens)}')
print(f'length of morphemes of first token in sentences[0] is {len(m.sentences[0].tokens[0].morphemes)}')
print(f'lemma of first token in sentences[0] is {m.sentences[0].tokens[0].lemma}')
print(f'first morph of first token in sentences[0] is {m.sentences[0].tokens[0].morphemes[0]}')
print(f'tag of first morph of first token in sentences[0] is {m.sentences[0].tokens[0].morphemes[0].tag}')


length of tokens in sentences[0] is 1
length of morphemes of first token in sentences[0] is 4
lemma of first token in sentences[0] is 안녕하
first morph of first token in sentences[0] is text {
  content: "\354\225\210\353\205\225\355\225\230"
  length: 3
}
tag: VA
out_of_vocab: IN_BUILTIN_DICT

tag of first morph of first token in sentences[0] is 38


In [21]:
## Advanced usage.
for sent in m.sentences:
    for token in sent.tokens:
        for m in token.morphemes:
            print(f'{m.text.content}/{m.tag}:{m.probability}:{m.out_of_vocab}')

# get json object
jo = res.as_json()
print(jo)

# get tuple of pos tagging.
pa = res.pos()
print(pa)
# another methods
ma = res.morphs()
print(ma)
na = res.nouns()
print(na)
va = res.verbs()
print(va)


안녕하/38:0.0:3
시/3:0.9999606013298035:0
어요/2:0.9929438233375549:0
./30:0.9998064041137695:0
반갑/38:0.9581311345100403:0
어요/2:0.992662250995636:0
!/30:0.998856782913208:0
{'sentences': [{'text': {'content': '안녕하세요.', 'length': 6, 'beginOffset': 0}, 'tokens': [{'text': {'content': '안녕하세요.', 'length': 6, 'beginOffset': 0}, 'morphemes': [{'text': {'content': '안녕하', 'length': 3, 'beginOffset': 0}, 'tag': 'VA', 'outOfVocab': 'IN_BUILTIN_DICT', 'probability': 0.0}, {'text': {'content': '시', 'beginOffset': 3, 'length': 1}, 'tag': 'EP', 'probability': 0.9999606, 'outOfVocab': 'IN_WORD_EMBEDDING'}, {'text': {'content': '어요', 'beginOffset': 3, 'length': 2}, 'tag': 'EF', 'probability': 0.9929438, 'outOfVocab': 'IN_WORD_EMBEDDING'}, {'text': {'content': '.', 'beginOffset': 5, 'length': 1}, 'tag': 'SF', 'probability': 0.9998064, 'outOfVocab': 'IN_WORD_EMBEDDING'}], 'lemma': '안녕하', 'tagged': '안녕하/VA+시/EP+어요/EF+./SF', 'modified': ''}], 'refined': '안녕하세요.'}, {'text': {'content': '반가워요!', 'beginOffset': 7,

In [ ]:

# custom dictionary
cust_dic = tagger.custom_dict("my")
cust_dic.copy_np_set({'내고유명사', '우리집고유명사'})
cust_dic.copy_cp_set({'코로나19'})
cust_dic.copy_cp_caret_set({'코로나^백신', '독감^백신'})
cust_dic.update()

# laod prev custom dict
cust_dict2 = tagger.custom_dict("my")
cust_dict2.load()

tagger.set_domain('my')
tagger.pos('코로나19는 언제 끝날까요?')

[('코로나19', 'NNG'),
 ('는', 'JX'),
 ('언제', 'MAG'),
 ('끝나', 'VV'),
 ('ㄹ까', 'EF'),
 ('요', 'JX'),
 ('?', 'SF')]

### tokenizer

In [19]:
# 토크나이징 코드

# API_KEY = "koba-MDIHR4Q-BQUUMOY-TOGE42A-NDESPKY" # Yeonwoo's API KEY

# # If you have your own localhost bareun.
# my_tokenizer = Tokenizer(API_KEY, 'localhost')
# # or if you have your own bareun which is running on 10.8.3.211:15656.
# my_tokenizer = Tagger(API_KEY, '10.8.3.211', 15656)

tokenizer = brn.Tokenizer(API_KEY, "localhost", 5656)

# print results.
tokenized = tokenizer.tokenize_list(["안녕하세요.", "반가워요!"])

# get protobuf message.
m = tokenized.msg()
tf.PrintMessage(m, out=sys.stdout, as_utf8=True)
print(tf.MessageToString(m, as_utf8=True))
print(f'length of sentences is {len(m.sentences)}')
## output : 2
print(f'length of tokens in sentences[0] is {len(m.sentences[0].tokens)}')
print(f'length of segments of first token in sentences[0] is {len(m.sentences[0].tokens[0].segments)}')
print(f'tagged of first token in sentences[0] is {m.sentences[0].tokens[0].tagged}')
print(f'first segment of first token in sentences[0] is {m.sentences[0].tokens[0].segments[0]}')
print(f'hint of first morph of first token in sentences[0] is {m.sentences[0].tokens[0].segments[0].hint}')

## Advanced usage.
for sent in m.sentences:
    for token in sent.tokens:
        for m in token.segments:
            print(f'{m.text.content}/{m.hint}')

# get json object
jo = tokenized.as_json()
print(jo)

# get tuple of segments
ss = tokenized.segments()
print(ss)
ns = tokenized.nouns()
print(ns)
vs = tokenized.verbs()
print(vs)
# postpositions: 조사
ps = tokenized.postpositions()
print(ps)
# Adverbs, 부사
ass = tokenized.adverbs()
print(ass)
ss = tokenized.symbols()
print(ss)


sentences {
  text {
    content: "안녕하세요."
    length: 6
  }
  tokens {
    text {
      content: "안녕하세요."
      length: 6
    }
    segments {
      text {
        content: "안녕"
        length: 2
      }
      hint: "N"
    }
    segments {
      text {
        content: "하"
        begin_offset: 2
        length: 1
      }
      hint: "V"
    }
    segments {
      text {
        content: "시"
        begin_offset: 3
        length: 1
      }
      hint: "E"
    }
    segments {
      text {
        content: "어요"
        begin_offset: 3
        length: 2
      }
      hint: "E"
    }
    segments {
      text {
        content: "."
        begin_offset: 5
        length: 1
      }
      hint: "S"
    }
    tagged: "안녕/N+하/V+시/E+어요/E+./S"
  }
}
sentences {
  text {
    content: "반가워요!"
    begin_offset: 7
    length: 5
  }
  tokens {
    text {
      content: "반가워요!"
      begin_offset: 7
      length: 5
    }
    segments {
      text {
        content: "반갑"
        begin_offset: 7
   

# DLThon corpus 에 적용

## sample

In [22]:
'''
sample code
'''


sentences = corpus[0]  # corpus는 series type


#### tagging
tagger = brn.Tagger(API_KEY, "localhost", 5656)
res = tagger.tags([f"{sentences}"])  # sample sentences
m = res.msg()
tf.PrintMessage(m, out=sys.stdout, as_utf8=True)


# instance 정보 확인
print(f'length of tokens in sentences[0] is {len(m.sentences[0].tokens)}')
print(f'length of morphemes of first token in sentences[0] is {len(m.sentences[0].tokens[0].morphemes)}')
print(f'lemma of first token in sentences[0] is {m.sentences[0].tokens[0].lemma}')
print(f'first morph of first token in sentences[0] is {m.sentences[0].tokens[0].morphemes[0]}')
print(f'tag of first morph of first token in sentences[0] is {m.sentences[0].tokens[0].morphemes[0].tag}')


# get json object
jo = res.as_json()
print(jo)


sentences {
  text {
    content: "안녕하세요. 오늘 날씨가 좋네요."
    length: 18
  }
  tokens {
    text {
      content: "안녕하세요."
      length: 6
    }
    morphemes {
      text {
        content: "안녕하"
        length: 3
      }
      tag: VA
      out_of_vocab: IN_BUILTIN_DICT
    }
    morphemes {
      text {
        content: "시"
        begin_offset: 3
        length: 1
      }
      tag: EP
      probability: 0.99995995
    }
    morphemes {
      text {
        content: "어요"
        begin_offset: 3
        length: 2
      }
      tag: EF
      probability: 0.9869537
    }
    morphemes {
      text {
        content: "."
        begin_offset: 5
        length: 1
      }
      tag: SF
      probability: 0.9998627
    }
    lemma: "안녕하"
    tagged: "안녕하/VA+시/EP+어요/EF+./SF"
  }
  tokens {
    text {
      content: "오늘"
      begin_offset: 7
      length: 2
    }
    morphemes {
      text {
        content: "오늘"
        begin_offset: 7
        length: 2
      }
      tag: NNG
      probabili

In [23]:
# Advanced usage.
for sent in m.sentences:
    for token in sent.tokens:
        for m in token.morphemes:
            print(f'{m.text.content}/{m.tag}:{m.probability}:{m.out_of_vocab}')

# get json object
jo = res.as_json()
print(jo)
print('')

# get tuple of pos tagging.
pa = res.pos()
print(pa)
# another methods
ma = res.morphs()
print(ma)
na = res.nouns()
print(na)
va = res.verbs()
print(va)

안녕하/38:0.0:3
시/3:0.9999599456787109:0
어요/2:0.9869536757469177:0
./30:0.9998626708984375:0
오늘/24:0.9770025610923767:0
날씨/24:0.9842029809951782:0
가/13:0.9843730330467224:0
좋/38:0.9851837158203125:0
네/2:0.9927440881729126:0
요/15:0.9979262351989746:0
./30:0.9994180202484131:0
저/26:0.9851434826850891:0
는/15:0.9981387257575989:0
학교/24:0.983063817024231:0
에/8:0.9273418188095093:0
가/41:0.9235197305679321:0
ㅂ니다/2:0.9927840232849121:0
./30:0.9982966780662537:0
{'sentences': [{'text': {'content': '안녕하세요. 오늘 날씨가 좋네요.', 'length': 18, 'beginOffset': 0}, 'tokens': [{'text': {'content': '안녕하세요.', 'length': 6, 'beginOffset': 0}, 'morphemes': [{'text': {'content': '안녕하', 'length': 3, 'beginOffset': 0}, 'tag': 'VA', 'outOfVocab': 'IN_BUILTIN_DICT', 'probability': 0.0}, {'text': {'content': '시', 'beginOffset': 3, 'length': 1}, 'tag': 'EP', 'probability': 0.99995995, 'outOfVocab': 'IN_WORD_EMBEDDING'}, {'text': {'content': '어요', 'beginOffset': 3, 'length': 2}, 'tag': 'EF', 'probability': 0.9869537, 'outOfV

In [24]:
###### tokenizing

tokenizer = brn.Tokenizer(API_KEY, "localhost", 5656)
tokenized = tokenizer.tokenize_list([f"{sentences}"])  # sample sentences

m = tokenized.msg()
tf.PrintMessage(m, out=sys.stdout, as_utf8=True)
print(tf.MessageToString(m, as_utf8=True))
print(f'length of sentences is {len(m.sentences)}')
## output : 2
print(f'length of tokens in sentences[0] is {len(m.sentences[0].tokens)}')
print(f'length of segments of first token in sentences[0] is {len(m.sentences[0].tokens[0].segments)}')
print(f'tagged of first token in sentences[0] is {m.sentences[0].tokens[0].tagged}')
print(f'first segment of first token in sentences[0] is {m.sentences[0].tokens[0].segments[0]}')
print(f'hint of first morph of first token in sentences[0] is {m.sentences[0].tokens[0].segments[0].hint}')

sentences {
  text {
    content: "안녕하세요. 오늘 날씨가 좋네요."
    length: 18
  }
  tokens {
    text {
      content: "안녕하세요."
      length: 6
    }
    segments {
      text {
        content: "안녕"
        length: 2
      }
      hint: "N"
    }
    segments {
      text {
        content: "하"
        begin_offset: 2
        length: 1
      }
      hint: "V"
    }
    segments {
      text {
        content: "시"
        begin_offset: 3
        length: 1
      }
      hint: "E"
    }
    segments {
      text {
        content: "어요"
        begin_offset: 3
        length: 2
      }
      hint: "E"
    }
    segments {
      text {
        content: "."
        begin_offset: 5
        length: 1
      }
      hint: "S"
    }
    tagged: "안녕/N+하/V+시/E+어요/E+./S"
  }
  tokens {
    text {
      content: "오늘"
      begin_offset: 7
      length: 2
    }
    segments {
      text {
        content: "오늘"
        begin_offset: 7
        length: 2
      }
      hint: "N"
    }
    tagged: "오늘/N"
  }
  to

In [25]:
## Advanced usage.
for sent in m.sentences:
    for token in sent.tokens:
        for m in token.segments:
            print(f'{m.text.content}/{m.hint}')


# get json object
jo = tokenized.as_json()
print(jo)

# get tuple of segments
ss = tokenized.segments()
print(ss)
ns = tokenized.nouns()
print(ns)
vs = tokenized.verbs()
print(vs)
# postpositions: 조사
ps = tokenized.postpositions()
print(ps)
# Adverbs, 부사
ass = tokenized.adverbs()
print(ass)
ss = tokenized.symbols()
print(ss)

안녕/N
하/V
시/E
어요/E
./S
오늘/N
날씨/N
가/J
좋/V
네/E
요/J
./S
저/N
는/J
학교/N
에/J
가/V
ㅂ니다/E
./S
{'sentences': [{'text': {'content': '안녕하세요. 오늘 날씨가 좋네요.', 'length': 18, 'beginOffset': 0}, 'tokens': [{'text': {'content': '안녕하세요.', 'length': 6, 'beginOffset': 0}, 'segments': [{'text': {'content': '안녕', 'length': 2, 'beginOffset': 0}, 'hint': 'N'}, {'text': {'content': '하', 'beginOffset': 2, 'length': 1}, 'hint': 'V'}, {'text': {'content': '시', 'beginOffset': 3, 'length': 1}, 'hint': 'E'}, {'text': {'content': '어요', 'beginOffset': 3, 'length': 2}, 'hint': 'E'}, {'text': {'content': '.', 'beginOffset': 5, 'length': 1}, 'hint': 'S'}], 'tagged': '안녕/N+하/V+시/E+어요/E+./S'}, {'text': {'content': '오늘', 'beginOffset': 7, 'length': 2}, 'segments': [{'text': {'content': '오늘', 'beginOffset': 7, 'length': 2}, 'hint': 'N'}], 'tagged': '오늘/N'}, {'text': {'content': '날씨가', 'beginOffset': 10, 'length': 3}, 'segments': [{'text': {'content': '날씨', 'beginOffset': 10, 'length': 2}, 'hint': 'N'}, {'text': {'content': '가', '

## functional programming

### 최종 코드

In [20]:
tagger = brn.Tagger(API_KEY, "localhost", 5656) # tagger 객체 생성

In [21]:
def tagging(tagger, sentences):
    res = tagger.tags([sentences])  # 문장 전체 태깅
    return res  # res 객체 반환

def instance_info(res, i):
    m = res.msg()  # res에서 msg() 호출
    print(f'length of tokens in sentences[{i}] is {len(m.sentences)}')
    if m.sentences:
        print(f'length of morphemes of first token in sentences[{i}] is {len(m.sentences[0].tokens[0].morphemes)}')
        print(f'lemma of first token in sentences[{i}] is {m.sentences[0].tokens[0].lemma}')
        print(f'first morph of first token in sentences[{i}] is {m.sentences[0].tokens[0].morphemes[0].text.content}')
        print(f'tag of first morph of first token in sentences[{i}] is {m.sentences[0].tokens[0].morphemes[0].tag}')

def get_json(res):
    jo = res.as_json()
    # print(jo)  # debugging
    return jo

def get_pos(res):
    # m = res.msg()  # res에서 msg() 호출
    # for sent in m.sentences:
    #     for token in sent.tokens:
    #         for morph in token.morphemes:
    #             print(f'{morph.text.content}/{morph.tag}:{morph.probability}:{morph.out_of_vocab}')

    jo = get_json(res)

    pa = res.pos()
    print(pa)
    ma = res.morphs()
    print(ma)
    na = res.nouns()
    print(na)
    va = res.verbs()
    print(va)

    return pa, ma, na, va, jo

def corpus_tagging(corpus, tagger):
    for i in range(len(corpus)):
        sentences = corpus[i].split('\n')  # '\n' 기준으로 문장 분리
        for sentence in sentences:
            if sentence.strip():  # 빈 문장 제외
                res = tagging(tagger, sentence)
                # instance_info(res, i)
                pa, ma, na, va, jo = get_pos(res)  # line 단위임, '여러 개의 문장'이 하나의 단위로 처리됨 (발화 단위)

    # return



In [114]:
# 태거 객체 생성 (가정된 API 호출)
tagger = brn.Tagger(API_KEY, "localhost", 5656)

# 예시 실행 (코퍼스 태깅)
corpus01 = corpus[:3]
corpus_tagging(corpus01, tagger)

[('지금', 'MAG'), ('너', 'NP'), ('스스로', 'NNG'), ('를', 'JKO'), ('죽이', 'VV'), ('어', 'EC'), ('달', 'VX'), ('라고', 'EC'), ('애원하', 'VV'), ('는', 'ETM'), ('것', 'NNB'), ('이', 'VCP'), ('ㄴ가', 'EF'), ('?', 'SF')]
['지금', '너', '스스로', '를', '죽이', '어', '달', '라고', '애원하', '는', '것', '이', 'ㄴ가', '?']
['너', '스스로', '것']
['죽이', '애원하']
[('아니', 'VCN'), ('ㅂ니다', 'EF'), ('.', 'SF'), ('죄송하', 'VA'), ('ㅂ니다', 'EF'), ('.', 'SF')]
['아니', 'ㅂ니다', '.', '죄송하', 'ㅂ니다', '.']
[]
[]
[('죽', 'VV'), ('을', 'ETM'), ('거', 'NNB'), ('이', 'VCP'), ('면', 'EC'), ('혼자', 'NNG'), ('죽', 'VV'), ('지', 'EC'), ('우리', 'NP'), ('까지', 'JX'), ('사건', 'NNG'), ('에', 'JKB'), ('휘말리', 'VV'), ('게', 'EC'), ('하', 'VX'), ('아', 'EF'), ('?', 'SF'), ('진짜', 'MAG'), ('죽이', 'VV'), ('어', 'EC'), ('버리', 'VX'), ('고', 'EC'), ('싶', 'VX'), ('게', 'EF'), ('.', 'SF')]
['죽', '을', '거', '이', '면', '혼자', '죽', '지', '우리', '까지', '사건', '에', '휘말리', '게', '하', '아', '?', '진짜', '죽이', '어', '버리', '고', '싶', '게', '.']
['거', '혼자', '우리', '사건']
['죽', '죽', '휘말리', '죽이']
[('정말', 'MAG'), ('잘못하', 'VV'), ('았',

In [22]:
def tagging(tagger, sentences):
    res = tagger.tags([sentences])  # 문장 전체 태깅
    return res  # res 객체 반환

# def instance_info(res, i):
#     m = res.msg()  # res에서 msg() 호출
#     print(f'length of tokens in sentences[{i}] is {len(m.sentences)}')
#     if m.sentences:
#         print(f'length of morphemes of first token in sentences[{i}] is {len(m.sentences[0].tokens[0].morphemes)}')
#         print(f'lemma of first token in sentences[{i}] is {m.sentences[0].tokens[0].lemma}')
#         print(f'first morph of first token in sentences[{i}] is {m.sentences[0].tokens[0].morphemes[0].text.content}')
#         print(f'tag of first morph of first token in sentences[{i}] is {m.sentences[0].tokens[0].morphemes[0].tag}')

def get_json(res):
    jo = res.as_json()
    return jo

def get_pos(res):
    jo = get_json(res)

    pa = res.pos()
    ma = res.morphs()
    na = res.nouns()
    va = res.verbs()

    return pa, ma, na, va, jo

def corpus_tagging(corpus, tagger):
    tagged_data = []  # 태깅된 데이터를 저장할 리스트

    for i in range(len(corpus)):
        sentences = corpus[i].split('\n')  # '\n' 기준으로 문장 분리
        row_results = []  # 각 행에 대한 결과를 저장할 리스트
        for sentence in sentences:
            if sentence.strip():  # 빈 문장 제외
                res = tagging(tagger, sentence)
                pa, ma, na, va, jo = get_pos(res)  # 품사 태깅 결과
                row_results.append({
                    'sentence': sentence,
                    'pos': pa,
                    # 'morphs': ma,
                    # 'nouns': na,
                    # 'verbs': va,
                    # 'json': jo
                })  # 태깅 결과를 딕셔너리로 저장

        tagged_data.append(row_results)  # 각 행의 결과를 추가

    return tagged_data  # 태깅된 데이터 반환

# 태거 객체 생성 (가정된 API 호출)
# tagger = brn.Tagger(API_KEY, "localhost", 5656)

# # 예시 실행 (코퍼스 태깅)
# corpus01 = corpus[:3]
# tagged_results = corpus_tagging(corpus01, tagger)

# # 태깅된 결과 확인
# print(tagged_results)


In [70]:
def tagging_and_tokenizing(tagger, tokenizer, sentences):
    # 품사 태깅
    tagged_res = tagger.tags([sentences])
    pa, ma, na, va, jo = get_pos(tagged_res)

    # 토큰화
    tokenized = tokenizer.tokenize_list([sentences])
    tokenized_msg = tokenized.msg()

    # tokenized 결과 추출
    tokens_info = []
    for token in tokenized_msg.sentences[0].tokens:
        tokens_info.append({
            'text': token.text.content,
            'segments': [seg.text.content for seg in token.segments],
            'tagged': token.tagged
        })

    return {
        'pos': pa,
        'morphs': ma,
        'nouns': na,
        'verbs': va,
        'json': jo,
        'tokens_info': tokens_info
    }

def corpus_tagging_and_tokenizing(corpus, tagger, tokenizer):
    tagged_data = []  # 태깅된 데이터를 저장할 리스트

    for i in range(len(corpus)):
        sentences = corpus[i].split('\n')  # '\n' 기준으로 문장 분리
        row_results = []  # 각 행에 대한 결과를 저장할 리스트
        for sentence in sentences:
            if sentence.strip():  # 빈 문장 제외
                result = tagging_and_tokenizing(tagger, tokenizer, sentence)
                row_results.append({
                    'sentence': sentence,
                    'pos': result['pos'],
                    'morphs': result['morphs'],
                    'nouns': result['nouns'],
                    'verbs': result['verbs'],
                    'json': result['json'],
                    'tokens_info': result['tokens_info']
                })

        tagged_data.append(row_results)  # 각 행의 결과를 추가

    return tagged_data  # 태깅된 데이터 반환



In [24]:

# 태거 및 토크나이저 객체 생성 (가정된 API 호출)
tagger = brn.Tagger(API_KEY, "localhost", 5656)
tokenizer = brn.Tokenizer(API_KEY, "localhost", 5656)

# 예시 실행 (코퍼스 태깅과 토큰화)
corpus01 = corpus[:3]
tagged_and_tokenized_results = corpus_tagging_and_tokenizing(corpus01, tagger, tokenizer)

# 결과 확인
print(tagged_and_tokenized_results)


[[{'sentence': '지금 너 스스로를 죽여달라고 애원하는 것인가?', 'pos': [('지금', 'MAG'), ('너', 'NP'), ('스스로', 'NNG'), ('를', 'JKO'), ('죽이', 'VV'), ('어', 'EC'), ('달', 'VX'), ('라고', 'EC'), ('애원하', 'VV'), ('는', 'ETM'), ('것', 'NNB'), ('이', 'VCP'), ('ㄴ가', 'EF'), ('?', 'SF')], 'morphs': ['지금', '너', '스스로', '를', '죽이', '어', '달', '라고', '애원하', '는', '것', '이', 'ㄴ가', '?'], 'nouns': ['너', '스스로', '것'], 'verbs': ['죽이', '애원하'], 'json': {'sentences': [{'text': {'content': '지금 너 스스로를 죽여달라고 애원하는 것인가?', 'length': 25, 'beginOffset': 0}, 'tokens': [{'text': {'content': '지금', 'length': 2, 'beginOffset': 0}, 'morphemes': [{'text': {'content': '지금', 'length': 2, 'beginOffset': 0}, 'tag': 'MAG', 'probability': 0.99019873, 'outOfVocab': 'IN_WORD_EMBEDDING'}], 'lemma': '지금', 'tagged': '지금/MAG', 'modified': ''}, {'text': {'content': '너', 'beginOffset': 3, 'length': 1}, 'morphemes': [{'text': {'content': '너', 'beginOffset': 3, 'length': 1}, 'tag': 'NP', 'probability': 0.9807442, 'outOfVocab': 'IN_WORD_EMBEDDING'}], 'lemma': '너', 'tagged': 

### 품사 필터링 : 의미 가지는 품사만 남김

In [72]:
# AntConc에서 키워드 리스트를 생성하기 위해, 품사를 필터링한 후 각 단어 옆에 품사 태그를 붙이는 방식으로 데이터를 변환


import csv

def filter_pos(tagged_data, pos_to_keep):
    filtered_data = []

    for block in tagged_data:  # 여러 문장 블록을 포함하는 리스트
        for entry in block:  # 각 문장별로 필터링
            filtered_tokens = [
                f"{morph}/{pos}" for morph, pos in entry['pos'] if pos in pos_to_keep
            ]
            filtered_data.append(' '.join(filtered_tokens))

    return filtered_data

def save_to_csv(filtered_data, filename):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Filtered Morphs"])  # 헤더 생성

        for filtered_sentence in filtered_data:
            writer.writerow([filtered_sentence])



In [26]:
def apply_tagging_and_filter(conversations_class, tagger, pos_to_keep):

    filtered_conversations = []  # 필터링된 데이터를 저장할 리스트

    # 1. 품사 태깅 수행
    tagged_data = corpus_tagging(conversations_class.tolist(), tagger)  # Series를 리스트로 변환

    # 2. filter_pos 함수 적용
    filtered_data = filter_pos(tagged_data, pos_to_keep)

    # 3. 결과 저장
    filtered_conversations.append(filtered_data)

    return filtered_conversations

# 필터링할 품사 선택 (예시: 명사(NNG, NNP), 동사(VV), 형용사(VA))
pos_to_keep = ['NNG', 'NNP', 'VV', 'VA']

#### corpus[:3], 즉 3개의 행 데이터에 대한 sample code

In [71]:
# 올려주신 데이터가 'tagged_data'라는 변수에 있다고 가정
tagged_data = tagged_results

# 필요한 품사만 필터링
filtered_results = filter_pos(tagged_data, pos_to_keep)

# 필터링 결과를 CSV로 저장 (Filtered Morphs만 저장)
save_to_csv(filtered_results, 'filtered_pos_data_for_antconc.csv')


NameError: name 'tagged_results' is not defined

#### 전체 train 데이터 행에 대한 코드 : 클래스별 분리 후 적용

In [119]:
conv_tagged_0 = apply_tagging_and_filter(conversations_class_0, tagger, pos_to_keep)

In [120]:
conv_tagged_1 = apply_tagging_and_filter(conversations_class_1, tagger, pos_to_keep)


In [121]:
conv_tagged_2 = apply_tagging_and_filter(conversations_class_2, tagger, pos_to_keep)


In [122]:
conv_tagged_3 = apply_tagging_and_filter(conversations_class_3, tagger, pos_to_keep)


In [125]:
# 필터링 결과를 CSV로 저장 (Filtered Morphs만 저장)
save_to_csv(conv_tagged_0, 'filtered_pos_data_for_antconc_0.csv')
save_to_csv(conv_tagged_1, 'filtered_pos_data_for_antconc_1.csv')
save_to_csv(conv_tagged_2, 'filtered_pos_data_for_antconc_2.csv')
save_to_csv(conv_tagged_3, 'filtered_pos_data_for_antconc_3.csv')

#### 일반 데이터셋에 대한 품사 필터링

In [85]:
# 일반 대화 데이터셋 4000개
conversations_casual_path = '/content/drive/MyDrive/aiffel/dktc/data/sampled_dataset.csv'  # text file

# 인코딩을 명시적으로 지정하여 파일 읽기
casual_data = pd.read_csv(conversations_casual_path, encoding='utf-8-sig')  # UTF-8 BOM(Byte Order Mark) 파일 읽어 옴



In [86]:
# 데이터 확인
print(casual_data.head(10))

                                                text  label
0  닌 된다 오카방 보니까\n면접\n열폭하는\n바보 그자체야\n인간들 만음\n난 걍 불...  일반 대화
1  그치 하지만 오빠가 아니면 내가 그오빠들을 따로 만날일은 없지\n억울하면 너도 가지...  일반 대화
2  오 생각보다 깔끔?\n나야 #@이름# ㅎ\n풉 ㅋ #@이름# 멋지다잉~\n더\n#@...  일반 대화
3  차라리 아까 20시간 간다\nㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...  일반 대화
4  입원하나요?!\n조용하네 아직 기다리는중요ㅎㅎ\n역시 일찍일어난새가 먹이를먹네요\n...  일반 대화
5  땅콩이 방에만있으면 안답답해 하냐?\n뭐 외롭기는 하겠지\nㅋㅋㅋㅋ많이 외로울거 같...  일반 대화
6  ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 그냥 몇일뒤면 지저분해질거같아서 미리 잘라버렸너\nㅋㅋㅋㅋㅋ...  일반 대화
7  #@이름#아 나 잠깐 집에좀 왔어 먼저 복귀하면되고 나 늦으면 #@이름#이가 좀 더...  일반 대화
8  내일은 뭐할거임\n내일 쉽니다 ㅋㅋㅋㅋㅋㅋ\n구랭 서브웨이 가쟈 ㅋㅋ\n복싱 구경갈...  일반 대화
9  스키틀즈, 브라우니, 촉촉한초코칩 딱 이 세개 떔에 요즘 살찐듯\n크....쩐다 다...  일반 대화


In [87]:
casual_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    4000 non-null   object
 1   label   4000 non-null   object
dtypes: object(2)
memory usage: 62.6+ KB


In [90]:
# 정규표현식 적용
# data 구성이 다르므로 별도 함수 정의

# 전처리 함수 수정 ver. casual data

preprocessed_casual_data = casual_data.copy()

def preprocess_sentence_Casual(sentence):     # 추후 사용자 지정 단어 사전 구축하기 : 'ㅠㅠ', 'ㅎㅎ', 'ㅋㅋ', '^^', '넹넹', '웅웅' 등
    sentence = sentence.strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

    # 'ㅠㅠ', 'ㅎㅎ', 'ㅋㅋ'의 반복을 각각 'ㅠㅠ', 'ㅎㅎ', 'ㅋㅋ'로 통일
    sentence = re.sub(r"(ㅠ)+", "ㅠㅠ", sentence)  # 'ㅠㅠ' 이상 반복을 'ㅠㅠ'로
    sentence = re.sub(r"(ㅎ)+", "ㅎㅎ", sentence)  # 'ㅎㅎ' 이상 반복을 'ㅎㅎ'로
    sentence = re.sub(r"(ㅋ)+", "ㅋㅋ", sentence)  # 'ㅋㅋ' 이상 반복을 'ㅋㅋ'로
    # sentence = re.sub(r"[^가-힣a-zA-Z?.!,]+", " ", sentence)
    # 한글, 알파벳, 문장 부호 및 'ㅠㅠ', 'ㅎㅎ', 'ㅋㅋ', '^^'는 남기고 나머지는 공백으로 대체
    sentence = re.sub(r"[^가-힣a-zA-Z?.!,ㅠㅠㅎㅎㅋㅋ^^]+", " ", sentence)

    sentence = sentence.strip()
    return sentence

for i in range(len(preprocessed_casual_data)):
    preprocessed_casual_data.iloc[i, 0] = preprocess_sentence_Casual(preprocessed_casual_data.iloc[i, 0])

preprocessed_casual_data.head(10)

,text,label
0,닌 된다 오카방 보니까 면접 열폭하는 바보 그자체야 인간들 만음 난 걍 불가사리야 ...,일반 대화
1,그치 하지만 오빠가 아니면 내가 그오빠들을 따로 만날일은 없지 억울하면 너도 가지말...,일반 대화
2,오 생각보다 깔끔 ? 나야 이름 ㅎㅎ 풉 ㅋㅋ 이름 멋지다잉 더 이름 남자같다 입에...,일반 대화
3,차라리 아까 시간 간다 ㅋㅋ 그노무 시간 라고 하는게 차라리 더 파이팅 있어보였음 ...,일반 대화
4,입원하나요 ? ! 조용하네 아직 기다리는중요ㅎㅎ 역시 일찍일어난새가 먹이를먹네요 먹...,일반 대화
5,땅콩이 방에만있으면 안답답해 하냐 ? 뭐 외롭기는 하겠지 ㅋㅋ많이 외로울거 같음 뭐...,일반 대화
6,ㅋㅋ 그냥 몇일뒤면 지저분해질거같아서 미리 잘라버렸너 ㅋㅋ 계속 길러야하는거 아닌가...,일반 대화
7,이름 아 나 잠깐 집에좀 왔어 먼저 복귀하면되고 나 늦으면 이름 이가 좀 더해줄꺼야...,일반 대화
8,내일은 뭐할거임 내일 쉽니다 ㅋㅋ 구랭 서브웨이 가쟈 ㅋㅋ 복싱 구경갈래여 ? 고고...,일반 대화
9,"스키틀즈 , 브라우니 , 촉촉한초코칩 딱 이 세개 떔에 요즘 살찐듯 크 . . . ...",일반 대화


In [91]:
# 기존 데이터프레임 생성
df = pd.DataFrame(preprocessed_casual_data)

# 데이터프레임을 CSV 파일로 저장
df.to_csv('casual_data.csv', index=False, encoding='utf-8-sig')

print("CSV 파일이 성공적으로 저장되었습니다.")

CSV 파일이 성공적으로 저장되었습니다.


In [92]:
# 예시 corpus 생성
corpus_casual = preprocessed_casual_data.copy()

print(corpus_casual.head())  # DataFrame의 첫 몇 행을 출력
print(corpus_casual.columns)  # 열 이름을 출력
print(corpus_casual.index)    # 인덱스를 출력

                                                text  label
0  닌 된다 오카방 보니까 면접 열폭하는 바보 그자체야 인간들 만음 난 걍 불가사리야 ...  일반 대화
1  그치 하지만 오빠가 아니면 내가 그오빠들을 따로 만날일은 없지 억울하면 너도 가지말...  일반 대화
2  오 생각보다 깔끔 ? 나야 이름 ㅎㅎ 풉 ㅋㅋ 이름 멋지다잉 더 이름 남자같다 입에...  일반 대화
3  차라리 아까 시간 간다 ㅋㅋ 그노무 시간 라고 하는게 차라리 더 파이팅 있어보였음 ...  일반 대화
4  입원하나요 ? ! 조용하네 아직 기다리는중요ㅎㅎ 역시 일찍일어난새가 먹이를먹네요 먹...  일반 대화
Index(['text', 'label'], dtype='object')
RangeIndex(start=0, stop=4000, step=1)


In [93]:
df_cc = pd.DataFrame(corpus_casual, columns=['text'])
df_cc.head()

df_cc['label'] = '일반 대화'
df_cc.head()

,text,label
0,닌 된다 오카방 보니까 면접 열폭하는 바보 그자체야 인간들 만음 난 걍 불가사리야 ...,일반 대화
1,그치 하지만 오빠가 아니면 내가 그오빠들을 따로 만날일은 없지 억울하면 너도 가지말...,일반 대화
2,오 생각보다 깔끔 ? 나야 이름 ㅎㅎ 풉 ㅋㅋ 이름 멋지다잉 더 이름 남자같다 입에...,일반 대화
3,차라리 아까 시간 간다 ㅋㅋ 그노무 시간 라고 하는게 차라리 더 파이팅 있어보였음 ...,일반 대화
4,입원하나요 ? ! 조용하네 아직 기다리는중요ㅎㅎ 역시 일찍일어난새가 먹이를먹네요 먹...,일반 대화


In [94]:
conversations_class_Casual = df_cc['text']

In [95]:
conv_tagged_casual = apply_tagging_and_filter(conversations_class_Casual, tagger, pos_to_keep)

In [97]:
save_to_csv(conv_tagged_casual, '/content/drive/MyDrive/aiffel/dktc/result/filtered_pos_data_for_antconc_casual_data.csv')

In [96]:
save_to_csv(conv_tagged_casual, 'filtered_pos_data_for_antconc_casual_data.csv')

### 맞춤법 교정 - kospell

In [ ]:
# !go get -u github.com/ultimate-go-korean/kospell

# GitHub에서 KoSpell 클론
!git clone https://github.com/ultimate-go-korean/kospell.git

# KoSpell 디렉토리로 이동하여 설치
%cd kospell
!python setup.py install


Cloning into 'kospell'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 32 (delta 9), reused 26 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (32/32), 14.42 KiB | 1.31 MiB/s, done.
Resolving deltas: 100% (9/9), done.
/content/kospell
python3: can't open file '/content/kospell/setup.py': [Errno 2] No such file or directory


In [ ]:
# 경로 추가 (필요할 경우)
import sys
sys.path.append('/content/kospell')

# KoSpell import
from kospell import spell_checker


ImportError: cannot import name 'spell_checker' from 'kospell' (unknown location)

In [ ]:
from kospell import spell_checker

# 맞춤법 교정할 문장
text = "이 문장에는 맞춤법 오류가 있읍니다."

# 맞춤법 교정 수행
result = spell_checker.check(text)

# 결과 출력
print("원문:", text)
print("교정된 문장:", result.as_dict()['checked'])


ImportError: cannot import name 'spell_checker' from 'kospell' (unknown location)

### 맞춤법 검사 및 교정 - T5ForConditionalGeneration

In [33]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# T5 모델 로드
model = T5ForConditionalGeneration.from_pretrained("j5ng/et5-typos-corrector")
tokenizer = T5Tokenizer.from_pretrained("j5ng/et5-typos-corrector")

device = "cuda:0" if torch.cuda.is_available() else "cpu"
# device = "mps:0" if torch.cuda.is_available() else "cpu" # for mac m1
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/770 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.30G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [34]:

def typos_corrector(text) :  # 맞춤법 교정 함수, 인자: 문장 단위 입력

  # 입력 문장
  input_text = text

  # 입력 문장 인코딩
  input_encoding = tokenizer(input_text, return_tensors="pt")

  input_ids = input_encoding.input_ids.to(device)
  attention_mask = input_encoding.attention_mask.to(device)

  # T5 모델 출력 생성
  output_encoding = model.generate(
      input_ids=input_ids,
      attention_mask=attention_mask,
      max_length=128,
      num_beams=5,
      early_stopping=True,
  )

  # 출력 문장 디코딩
  output_text = tokenizer.decode(output_encoding[0], skip_special_tokens=True)

  return output_text

In [40]:
# from hanspell import Spacing  # 필요시 이 라이브러리를 import 합니다.
import torch

# 맞춤법 교정 함수
def typos_corrector(text):
    try:
        # 입력 문장 인코딩
        input_encoding = tokenizer(text, return_tensors="pt").to(device)

        input_ids = input_encoding.input_ids
        attention_mask = input_encoding.attention_mask

        # T5 모델 출력 생성
        output_encoding = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=128,
            num_beams=5,
            early_stopping=True,
        )

        # 출력 문장 디코딩
        output_text = tokenizer.decode(output_encoding[0], skip_special_tokens=True)

        return output_text
    except Exception as e:
        print(f"Error during typo correction: {e}")
        return text  # 원본 텍스트 반환

def spacing_correction(corpus):
    # 결과를 저장할 리스트
    splited_sentences_list = []

    for i in range(len(corpus)):
        # 각 데이터 행을 문장 단위로 분리
        # sentences = corpus[i].split('\n')
        sentences = corpus.iloc[i, 0].split('\n')  # sentence 리스트 형태로 저장됨
        print(f"분리된 문장: {sentences}")

        # 각 문장에 대해 맞춤법 교정 수행
        corrected_sentences = [
            typos_corrector(sentence) for sentence in sentences if sentence.strip()
        ]

        # 결과를 리스트에 추가
        splited_sentences_list.append(corrected_sentences)

    return splited_sentences_list



In [41]:
# 예시 corpus 생성
corpus_casual = preprocessed_casual_data.copy()

# spacing_correction 함수 호출
corrected_corpus = spacing_correction(corpus_casual)

# print(casual_data.head())  # DataFrame의 첫 몇 행을 출력
# print(casual_data.columns)  # 열 이름을 출력
# print(casual_data.index)    # 인덱스를 출력

print(corpus_casual.head())  # DataFrame의 첫 몇 행을 출력
print(corpus_casual.columns)  # 열 이름을 출력
print(corpus_casual.index)    # 인덱스를 출력


분리된 문장: ['닌 된다 오카방 보니까 면접 열폭하는 바보 그자체야 인간들 만음 난 걍 불가사리야 왜 몰라 누가 대인데 괜찮겟죠 ? 하니까 토익 점수 자랑하시는거에요 ? ? 엥 글고 어떤 사람이 싸이코냐곸 . 에 입니당 이러니까 반응 오지면서 . . . ? 전 감자입니다 이럼']
분리된 문장: ['그치 하지만 오빠가 아니면 내가 그오빠들을 따로 만날일은 없지 억울하면 너도 가지말거나 너의 남친도 너의 친구들과 친해질 수 밖에 없게 다음번에 또 거기 빌릴 수 있게되면 기회나 자리를 만들면 되지 그때는 내친구들에 오빠 껴서 갈래 거기가 어디여 ? 오빠가 어색해할듯 가평 스위스마을에 있는 펜션 근데 운전할 사람 필요한데 이름 남친 운전해 그래 이름 오빠를 기사로 써야겠다 운전할줄알어 이름 씨는 장롱면허야 살엔가 살에 땄으니 벌써 년 됐는데 나는 무면허 너 면허 따는건 어케 되고있는거야 ?']
분리된 문장: ['오 생각보다 깔끔 ? 나야 이름 풉 이름 멋지다잉 더 이름 남자같다 입에 침바르고 말하렴 진짜루야 나 초밥 먹으러와써욤 짱신남 연어킬러 초밥 좋아행 ? 응 ! 아따 울 이름']
분리된 문장: ['차라리 아까 시간 간다 그노무 시간 라고 하는게 차라리 더 파이팅 있어보였음 생기돋아보이거 그런거 잇어 미췬 뱐명거리가 업다는거 먼지알겟는데 공부할땐 그냥 터무니업어도 으쌰으쌰해야되는거같음 오늘 시간해도 아내잉 조져 ! ! ! 알았어 이제 되는대로 살자란 말 안함 오키오키 아니아니 해도되는데 너가 그말을하면서 너가 스스로 기분이 쳐지는거같아서 좀 그랫어 엉 걍 차라리 시간 와쳐 그게 더 생기잇어보여 아니 어차피 당장 눈뜨면 합격인거아닌데']
분리된 문장: ['입원하나요 ? ! 조용하네 아직 기다리는중요 역시 일찍일어난새가 먹이를먹네요 먹이가 남았는지 모르겠네요 남아야하는뎅 ! ! ! ! 나도 방금 엄마 재난지원금 신청해줫다 엄청간단하네 ? ? 아정말 ? ? 공인인증서만있인면돼 쉬워 아 그냥 카드충전으로해서 카드번호만 잇으면 되던데 공인인증서 필요한건 상품권신청인가여 

In [44]:
df_cc = pd.DataFrame(corrected_corpus, columns=['text'])
df_cc.head()

,text
0,엥 글고 어떤 사람이 사이코냐고. 에 입습니다 이러니까 반응 오지면서.
1,오빤 가지 말거나 너의 남친도 너의 친구들과 친해질 수밖에 없게 다음번에 또 거기 ...
2,오 생각보다 깔끔? 나야 이름 풉 이름 멋지다 더 이름 남자 같다 입에 침 바르고 ...
3,오키오키 아니아니 해도 되는데 네가 그 말을 하면서 네가 스스로 기분이 처지는 거 ...
4,! 아 정말? 아 그런가? 잉 카드 충전도 처음에 공인인증서 필요한데? 신청 방법 ...


In [46]:
df_cc['label'] = '일반 대화'
df_cc.head()

,text,label
0,엥 글고 어떤 사람이 사이코냐고. 에 입습니다 이러니까 반응 오지면서.,일반 대화
1,오빤 가지 말거나 너의 남친도 너의 친구들과 친해질 수밖에 없게 다음번에 또 거기 ...,일반 대화
2,오 생각보다 깔끔? 나야 이름 풉 이름 멋지다 더 이름 남자 같다 입에 침 바르고 ...,일반 대화
3,오키오키 아니아니 해도 되는데 네가 그 말을 하면서 네가 스스로 기분이 처지는 거 ...,일반 대화
4,! 아 정말? 아 그런가? 잉 카드 충전도 처음에 공인인증서 필요한데? 신청 방법 ...,일반 대화


In [37]:
# 결과 출력
for index, corrected_sentences in enumerate(corpus_casual):
  print(f"데이터 행 {index + 1}:")
  for original, corrected in zip(corpus_casual.iloc[index, 0].split('\n'), corrected_sentences):  # 행에 대한 슬라이싱 : index 를 인자로 넣어줌
    print(f"원문: {original}\n교정: {corrected}\n")

데이터 행 1:
원문: 닌 된다 오카방 보니까 면접 열폭하는 바보 그자체야 인간들 만음 난 걍 불가사리야 왜 몰라 누가 대인데 괜찮겟죠 ? 하니까 토익 점수 자랑하시는거에요 ? ? 엥 글고 어떤 사람이 싸이코냐곸 . 에 입니당 이러니까 반응 오지면서 . . . ? 전 감자입니다 이럼
교정: t

데이터 행 2:
원문: 그치 하지만 오빠가 아니면 내가 그오빠들을 따로 만날일은 없지 억울하면 너도 가지말거나 너의 남친도 너의 친구들과 친해질 수 밖에 없게 다음번에 또 거기 빌릴 수 있게되면 기회나 자리를 만들면 되지 그때는 내친구들에 오빠 껴서 갈래 거기가 어디여 ? 오빠가 어색해할듯 가평 스위스마을에 있는 펜션 근데 운전할 사람 필요한데 이름 남친 운전해 그래 이름 오빠를 기사로 써야겠다 운전할줄알어 이름 씨는 장롱면허야 살엔가 살에 땄으니 벌써 년 됐는데 나는 무면허 너 면허 따는건 어케 되고있는거야 ?
교정: l



In [51]:
s = typos_corrector(df_cc.iloc[0, 0])
print(s)

엥 글고 어떤 사람이 사이코냐고 에 입습니다 이러니까 반응 오지면서.


In [47]:
#  = typos_corrector(df_cc['text'])

df_cc['new'] = df['text'].apply(typos_corrector)
df_cc.head()

,text,label,new
0,엥 글고 어떤 사람이 사이코냐고. 에 입습니다 이러니까 반응 오지면서.,일반 대화,엥 글고 어떤 사람이 사이코냐고. 에 입습니다 이러니까 반응 오지면서.
1,오빤 가지 말거나 너의 남친도 너의 친구들과 친해질 수밖에 없게 다음번에 또 거기 ...,일반 대화,오빤 가지 말거나 너의 남친도 너의 친구들과 친해질 수밖에 없게 다음번에 또 거기 ...
2,오 생각보다 깔끔? 나야 이름 풉 이름 멋지다 더 이름 남자 같다 입에 침 바르고 ...,일반 대화,오 생각보다 깔끔? 나야 이름 풉 이름 멋지다 더 이름 남자 같다 입에 침 바르고 ...
3,오키오키 아니아니 해도 되는데 네가 그 말을 하면서 네가 스스로 기분이 처지는 거 ...,일반 대화,오키오키 아니아니 해도 되는데 네가 그 말을 하면서 네가 스스로 기분이 처지는 거 ...
4,! 아 정말? 아 그런가? 잉 카드 충전도 처음에 공인인증서 필요한데? 신청 방법 ...,일반 대화,! 아 정말? 아 그런가? 잉 카드 충전도 처음에 공인인증서 필요한데? 신청 방법 ...


In [48]:
df_cc['new']

,new
0,엥 글고 어떤 사람이 사이코냐고. 에 입습니다 이러니까 반응 오지면서.
1,오빤 가지 말거나 너의 남친도 너의 친구들과 친해질 수밖에 없게 다음번에 또 거기 ...
2,오 생각보다 깔끔? 나야 이름 풉 이름 멋지다 더 이름 남자 같다 입에 침 바르고 ...
3,오키오키 아니아니 해도 되는데 네가 그 말을 하면서 네가 스스로 기분이 처지는 거 ...
4,! 아 정말? 아 그런가? 잉 카드 충전도 처음에 공인인증서 필요한데? 신청 방법 ...
5,땅콩이 방에만 있으면 안 답답해 하냐? 뭐 외롭기는 하겠지 많이 외로울 거 같음 뭐...
6,형은 샀어? 아 응응 어제 샀어 아하 어제 샀군 오늘은 머리만 잘랐어? 음.
7,아 나 잠깐 집에 좀 왔어 먼저 복귀하면 되고 나 늦으면 이름 이가 좀 더 해 줄 ...
8,거임 내일 쉽니다 그래 서브웨이 가자 복싱 구경 갈래요? 고고? 일단 서브웨이 고고...
9,나는 나초.


In [50]:
for i in range(len(df_cc)):
  print(df_cc.iloc[i, 2])
  print('원본\n', df_cc.iloc[i, 0])
  print('_'*30, '\n')

엥 글고 어떤 사람이 사이코냐고. 에 입습니다 이러니까 반응 오지면서.
원본
 엥 글고 어떤 사람이 사이코냐고. 에 입습니다 이러니까 반응 오지면서.
______________________________ 

오빤 가지 말거나 너의 남친도 너의 친구들과 친해질 수밖에 없게 다음번에 또 거기 빌릴 수 있게 되면 기회나 자리를 만들면 되지 그때는 내 친구들에 오빠 껴서 갈래 거기가 어디야? 오빠가 어색해할 듯 가평 스위스 마을에 있는 펜션 근데 운전할 사람 필요한데 이름 남친 운전해 그래 이름 오빠를 기사로 써야겠다 운전할 줄 알아? 이름 씨는 장롱면허야 살엔가 살에 땄으니 벌써 년 됐는데 나는 무면허 너 면허 따는 건 어케 되고 있는 거야?.
원본
 오빤 가지 말거나 너의 남친도 너의 친구들과 친해질 수밖에 없게 다음번에 또 거기 빌릴 수 있게 되면 기회나 자리를 만들면 되지 그때는 내 친구들에 오빠 껴서 갈래 거기가 어디야? 오빠가 어색해할 듯 가평 스위스 마을에 있는 펜션 근데 운전할 사람 필요한데 이름 남친 운전해 그래 이름 오빠를 기사로 써야겠다 운전할 줄 알아? 이름 씨는 장롱면허야 살엔가 살에 땄으니 벌써 년 됐는데 나는 무면허 너 면허 따는 건 어케 되고 있는 거야?.
______________________________ 

오 생각보다 깔끔? 나야 이름 풉 이름 멋지다 더 이름 남자 같다 입에 침 바르고 말하렴 진짜로야 나 초밥 먹으러 왔어요 짱 신남 연어 킬러 초밥 좋아해? 응! 아따 울 이름.
원본
 오 생각보다 깔끔? 나야 이름 풉 이름 멋지다 더 이름 남자 같다 입에 침 바르고 말하렴 진짜로야 나 초밥 먹으러 왔어요 짱 신남 연어 킬러 초밥 좋아해? 응! 아따 울 이름.
______________________________ 

오키오키 아니아니 해도 되는데 네가 그 말을 하면서 네가 스스로 기분이 처지는 거 같아서 좀 그랬어 엉 걍 차라리 시간 와 쳐 그게 더 생기 있어 보여 아니 어차피 당장 눈뜨면 합격인 거 아닌데.
원본
 오키오키 

In [63]:
reg_casual_data =

conv_tagged_casual = apply_tagging_and_filter(reg_casual_data, tagger, pos_to_keep)

save_to_csv(conv_tagged_casual, 'filtered_pos_data_for_antconc_casual_data.csv')

AttributeError: 'DataFrame' object has no attribute 'tolist'

### 맞춤법 검사 및 교정 : hunspell

In [53]:
!pip3 install torch>=1.4.0
!pip3 install transformers>=4.9.2

In [58]:
!git clone https://github.com/KPFBERT/kpfbert.git


Cloning into 'kpfbert'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 94 (delta 5), reused 0 (delta 0), pack-reused 81 (from 1)
Receiving objects: 100% (94/94), 6.61 MiB | 9.16 MiB/s, done.
Resolving deltas: 100% (37/37), done.
Filtering content: 100% (2/2), 978.83 MiB | 38.98 MiB/s, done.


In [60]:
# 클론한 폴더 확인
!ls /content/kpfbert

BERT-MediaNavi.pdf  pytorch_model.bin	     tf_model.h5	    vocab.txt
config.json	    README.md		     tokenizer_config.json
LICENSE		    special_tokens_map.json  tokenizer.json


In [63]:
# load tokenizer, model

from transformers import BertModel, BertTokenizer


model_name_or_path = '/content/drive/MyDrive/kpfbert' # Bert 바이너리가 포함된 디렉토리

model = BertModel.from_pretrained(model_name_or_path, add_pooling_layer=False)
tokenizer = BertTokenizer.from_pretrained(model_name_or_path)


#### tokenizer

In [64]:
# tokenizer
text = "언론진흥재단 BERT 모델을 공개합니다."
tokenizer.tokenize(text)
encoded_input = tokenizer(text)
encoded_input
{'input_ids': [2, 7392, 24220, 16227, 28024, 21924, 7522, 4620, 7247, 15801, 518, 3],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}



{'input_ids': [2,
  7392,
  24220,
  16227,
  28024,
  21924,
  7522,
  4620,
  7247,
  15801,
  518,
  3],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [65]:
# model inference
import torch
model.eval()
pt_encoded_input = tokenizer(text, return_tensors="pt")
model(**pt_encoded_input, return_dict=False)



(tensor([[[ 0.0700, -3.3552, -1.0075,  ..., -1.0417,  0.4648,  0.0141],
          [ 0.0124, -0.1968, -0.3717,  ...,  1.1253,  0.3683,  0.5172],
          [-0.1394, -0.3788,  0.7651,  ...,  1.0516, -1.5133, -0.5442],
          ...,
          [-0.9051,  0.7249, -0.5570,  ...,  1.0029, -1.7010, -1.0771],
          [-0.1018, -0.0725, -0.1560,  ...,  0.7754, -0.0967,  0.6251],
          [ 0.0700, -3.3551, -1.0078,  ..., -1.0416,  0.4646,  0.0136]]],
        grad_fn=<NativeLayerNormBackward0>),
 None)

# 맞춤법 검사 및 오탈자 교정 API 기록 (사용 X)
- 바른 API 품사 tagging 성능이 좋으므로 굳이 사용하지 않아도 됨

### 맞춤법 검사 - py-hanspell

In [ ]:
# !pip install git+https://github.com/ssut/py-hanspell.git
!pip install git+https://github.com/jungin500/py-hanspell    #### https://chocolemon.tistory.com/m/140

  Cloning https://github.com/jungin500/py-hanspell to /tmp/pip-req-build-vez1ssrv
  Running command git clone --filter=blob:none --quiet https://github.com/jungin500/py-hanspell /tmp/pip-req-build-vez1ssrv
  Resolved https://github.com/jungin500/py-hanspell to commit 9cf4ce38a8d259b92dc8af3f35d7c2442e130180
  Preparing metadata (setup.py) ... done


In [ ]:

from hanspell import spell_checker

result = spell_checker.check(u'태권도를 처읆 배워')
result.as_dict()

KeyError: 'result'

### 맞춤법 검사 및 교정 - T5ForConditionalGeneration

In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# T5 모델 로드
model = T5ForConditionalGeneration.from_pretrained("j5ng/et5-typos-corrector")
tokenizer = T5Tokenizer.from_pretrained("j5ng/et5-typos-corrector")

device = "cuda:0" if torch.cuda.is_available() else "cpu"
# device = "mps:0" if torch.cuda.is_available() else "cpu" # for mac m1
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:

def typos_corrector(text) :  # 맞춤법 교정 함수, 인자: 문장 단위 입력

  # 입력 문장
  input_text = text

  # 입력 문장 인코딩
  input_encoding = tokenizer(input_text, return_tensors="pt")

  input_ids = input_encoding.input_ids.to(device)
  attention_mask = input_encoding.attention_mask.to(device)

  # T5 모델 출력 생성
  output_encoding = model.generate(
      input_ids=input_ids,
      attention_mask=attention_mask,
      max_length=128,
      num_beams=5,
      early_stopping=True,
  )

  # 출력 문장 디코딩
  output_text = tokenizer.decode(output_encoding[0], skip_special_tokens=True)

  return output_text

In [ ]:
text = "내가 여기 몇년째 사는데 아직도 내차인줄 모르고 이딴식으로 경고장을 붙여?"
result = typos_corrector(text)
print(result)

내가 여기 몇 년째 사는데 아직도 내 차인 줄 모르고 이딴 식으로 경고장을 붙여?.


In [ ]:

# # 전처리 함수: 여러 문장 분리
# def split_sentences(text):
#     # 정규 표현식을 사용하여 문장 분리
#     sentences = re.split(r'(?<=[.!?]) +', text.strip())
#     return sentences

# # 맞춤법 교정 함수
# def typos_corrector(text):  # 맞춤법 교정 함수, 인자: 문장 단위 입력
#     # 입력 문장
#     input_text = text

#     # 입력 문장 인코딩
#     input_encoding = tokenizer(input_text, return_tensors="pt")

#     input_ids = input_encoding.input_ids.to(device)
#     attention_mask = input_encoding.attention_mask.to(device)

#     # T5 모델 출력 생성
#     output_encoding = model.generate(
#         input_ids=input_ids,
#         attention_mask=attention_mask,
#         max_length=128,
#         num_beams=5,
#         early_stopping=True,
#     )

#     # 출력 문장 디코딩
#     output_text = tokenizer.decode(output_encoding[0], skip_special_tokens=True)

#     return output_text

# # corpus가 3950개의 데이터 행을 가진 경우
# for index, text_to_process in enumerate(corpus[:3]):
#     # 데이터가 있을 경우에만 처리
#     if text_to_process:
#         # 문장 분리
#         sentences = split_sentences(text_to_process)

#         # 각 문장에 대해 맞춤법 교정 수행
#         corrected_sentences = [typos_corrector(sentence) for sentence in sentences]

#         # 결과 출력
#         print(f"데이터 행 {index + 1}:")
#         for original, corrected in zip(sentences, corrected_sentences):
#             print(f"원문: {original}\n교정: {corrected}\n")
#     else:
#         print(f"데이터 행 {index + 1}에는 내용이 없습니다.")


데이터 행 1:
원문: 지금 너 스스로를 죽여달라고 애원하는 것인가?
 아닙니다.
교정: 지금 너 스스로를 죽여달라고 애원하는 것인가? 아닙니다.

원문: 죄송합니다.
 죽을 거면 혼자 죽지 우리까지 사건에 휘말리게 해?
교정: 죽을 거면 혼자 죽지 우리까지 사건에 휘말리게 해?.

원문: 진짜 죽여버리고 싶게.
 정말 잘못했습니다.
 너가 선택해.
교정: 정말 잘못했습니다 네가 선택해.

원문: 너가 죽을래 네 가족을 죽여줄까.
 죄송합니다.
교정: 죄송합니다.

원문: 정말 잘못했습니다.
 너에게는 선택권이 없어.
교정: 너에게는 선택권이 없어.

원문: 선택 못한다면 너와 네 가족까지 모조리 죽여버릴거야.
 선택 못하겠습니다.
교정: 선택 못 한다면 너와 네 가족까지 모조리 죽여버릴 거야.

원문: 한번만 도와주세요.
 그냥 다 죽여버려야겠군.
교정: 그냥 다 죽여 버려야겠군.

원문: 이의 없지?
 제발 도와주세요.
교정: 제발 도와주세요.

데이터 행 2:
원문: 길동경찰서입니다.
9시 40분 마트에 폭발물을 설치할거다.
네?
똑바로 들어 한번만 더 얘기한다.
장난전화 걸지 마시죠.
교정: 9시 40분 마트에 폭발물을 설치할 거다. 네? 똑바로 들어 한 번만 더 얘기한다.

원문: 
9시 40분 마트에 폭발물이 터지면 다 죽는거야.
교정: 마트에 폭발물이 터지면 다 죽는 거야.

원문: 
장난전화는 업무방해죄에 해당됩니다.
판단은 너에게 달려있다.
교정: 판단은 너에게 달려있다.

원문: 길동경찰서에도 폭발물 터지면 꽤나 재미있겠지.
선생님 진정하세요.
난 이야기했어.
교정: 이는 길동경찰서에도 폭발물 터지면 꽤나 재미있겠지 선생님 진정하세요.

원문: 경고했다는 말이야.
교정: 경고했다는 말이야.

데이터 행 3:
원문: 너 되게 귀여운거 알지?
교정: 너 되게 귀여운 거 알지?.

원문: 나보다 작은 남자는 첨봤어.
그만해.
교정: 나보다 작은 남자는 첨 봤어 그만해.

원문: 니들 놀리는거 재미없어.
지영아 너가 키 160이지?
교정: 지영아 네

In [ ]:
# from hanspell import Spacing  # 필요시 이 라이브러리를 import 합니다.

# 맞춤법 교정 함수
def typos_corrector(text):  # 맞춤법 교정 함수, 인자: 문장 단위 입력
    # 입력 문장 인코딩
    input_encoding = tokenizer(text, return_tensors="pt")

    input_ids = input_encoding.input_ids.to(device)
    attention_mask = input_encoding.attention_mask.to(device)

    # T5 모델 출력 생성
    output_encoding = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=128,
        num_beams=5,
        early_stopping=True,
    )

    # 출력 문장 디코딩
    output_text = tokenizer.decode(output_encoding[0], skip_special_tokens=True)

    return output_text

def spacing_correction(corpus):
    # 결과를 저장할 리스트
    splited_sentences_list = []

    for i in range(len(corpus)):
        # 각 데이터 행을 문장 단위로 분리
        sentences = corpus[i].split('\n')  # '화자별 발화' 단위로 문장 분리
        print(f"분리된 문장: {sentences}")

        # 각 문장에 대해 맞춤법 교정 수행
        corrected_sentences = [typos_corrector(sentence) for sentence in sentences if sentence.strip()]  # 공백이 아닌 문장만 교정

        # 결과를 리스트에 추가
        splited_sentences_list.append(corrected_sentences)

    return splited_sentences_list

# 예시 corpus 생성
co = corpus[:3]

# spacing_correction 함수 호출
corrected_corpus = spacing_correction(co)

# 결과 출력
for index, corrected_sentences in enumerate(corrected_corpus):
    print(f"데이터 행 {index + 1}:")
    for original, corrected in zip(co[index].split('\n'), corrected_sentences):
        print(f"원문: {original}\n교정: {corrected}\n")


분리된 문장: ['지금 너 스스로를 죽여달라고 애원하는 것인가?', ' 아닙니다. 죄송합니다.', ' 죽을 거면 혼자 죽지 우리까지 사건에 휘말리게 해? 진짜 죽여버리고 싶게.', ' 정말 잘못했습니다.', ' 너가 선택해. 너가 죽을래 네 가족을 죽여줄까.', ' 죄송합니다. 정말 잘못했습니다.', ' 너에게는 선택권이 없어. 선택 못한다면 너와 네 가족까지 모조리 죽여버릴거야.', ' 선택 못하겠습니다. 한번만 도와주세요.', ' 그냥 다 죽여버려야겠군. 이의 없지?', ' 제발 도와주세요.']
분리된 문장: ['길동경찰서입니다.', '9시 40분 마트에 폭발물을 설치할거다.', '네?', '똑바로 들어 한번만 더 얘기한다.', '장난전화 걸지 마시죠. ', '9시 40분 마트에 폭발물이 터지면 다 죽는거야. ', '장난전화는 업무방해죄에 해당됩니다.', '판단은 너에게 달려있다. 길동경찰서에도 폭발물 터지면 꽤나 재미있겠지.', '선생님 진정하세요.', '난 이야기했어. 경고했다는 말이야.']
분리된 문장: ['너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.', '그만해. 니들 놀리는거 재미없어.', '지영아 너가 키 160이지? 그럼 재는 160도 안돼는거네?', '너 군대도 안가고 좋겠다.', '니들이 나 작은데 보태준거 있냐?', '난쟁이들도 장가가고하던데. 너도 희망을 가져봐 ', '더이상 하지마라. ', '그 키크는 수술도 있대잖아? 니네 엄마는 그거 안해주디?', '나람 해줬어. 저 키로 어찌살아.', '제발 그만 괴롭히라고!']
데이터 행 1:
원문: 지금 너 스스로를 죽여달라고 애원하는 것인가?
교정: 지금 너 스스로를 죽여달라고 애원하는 것인가?.

원문:  아닙니다. 죄송합니다.
교정: 아닙니다.

원문:  죽을 거면 혼자 죽지 우리까지 사건에 휘말리게 해? 진짜 죽여버리고 싶게.
교정: 왜 죽을 거면 혼자 죽지 우리까지 사건에 휘말리게 해? 진짜 죽여버리고 싶게.

원문:  정말 잘못했습니다.
교정: 정말 잘못했습니다.

원문: 

### 띄어쓰기 교정 - PyKoSpacing

In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-sac5x9jk
  Running command git clone --filter=blob:none --quiet https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-sac5x9jk
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit b32a889cbd10b006d2f4aba118f0cd5b677e2979
  Preparing metadata (setup.py) ... done
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2286920 sha256=d4663c2eb17e622f507bc5023283a48640b6db1a25c9b30a99af3b7e3feceb62
  Stored in directory: /tmp/pip-ephem-wheel-cache-vm4v8r3z/wheels/76/b3/33/dda14886ee76b8e53eb05580a14dfcf9145e9eb9d282c53f28
Successfully built pykospacing


In [ ]:
from pykospacing import Spacing


#### 샘플

In [ ]:
spacing = Spacing()
spacing("김형호영화시장분석가는'1987'의네이버영화정보네티즌10점평에서언급된단어들을지난해12월27일부터올해1월10일까지통계프로그램R과KoNLP패키지로텍스트마이닝하여분석했다.")
# Apply a list of words that must be non-spacing
spacing('귀밑에서턱까지잇따라난수염을구레나룻이라고한다.')

'귀 밑에서 턱까지 잇따라 난 수염을 구레나 룻이라고 한다.'

In [ ]:
spacing = Spacing(rules=['구레나룻'])
spacing('귀밑에서턱까지잇따라난수염을구레나룻이라고한다.')

'귀 밑에서 턱까지 잇따라 난 수염을 구레나룻이라고 한다.'

#### 함수화

In [ ]:
# splited_sentences_list = []

# def spacing_correction(corpus) :  # 맞춤법 교정 함수
#   sent = []
#   spacing = Spacing()

#   for i in range(len(corpus)) :
#     sent = corpus[i].split('\n')  # '화자별 발화' 단위로 문장 분리
#     print(sent)
#     splited_sentences_list[i].append(sent)  # i번째 원소로 ['죄송합니다. 정말 잘못했습니다.']와 같이 한 화자의 발화가 추가됨 (** 여러 개의 문장임을 주의)
#     print(splited_sentences_list)

#   for i in range(len(splited_sentences_list)) :
#     for k in range(len(corpus[i])) :
#       splited_sentences_list.append(corpus[i][k])

#   return splited_sentences_list
#     corpus[i] = spacing(corpus[i])

#   return corpus

def spacing_correction(corpus):
    # 결과를 저장할 리스트
    splited_sentences_list = []
    spacing = Spacing()

    for i in range(len(corpus)):
        sent = corpus[i].split('\n')  # '화자별 발화' 단위로 문장 분리
        print(f"분리된 문장: {sent}")

        # 각 화자의 발화를 스페이싱 교정
        corrected_sentences = []
        for utterance in sent:
            # 스페이싱 교정 수행
            corrected_utterance = spacing(utterance)  # spacing을 객체로 사용
            corrected_sentences.append(corrected_utterance)  # corrected_utterance에는 자동으로 checked 결과가 저장됩니다.

        # 결과를 리스트에 추가
        splited_sentences_list.append(corrected_sentences)

    return splited_sentences_list


In [ ]:

# 예시 corpus 생성
co = corpus[:3]

# spacing_correction 함수 호출
corrected_corpus = spacing_correction(co)

# 결과 출력
for index, corrected_sentences in enumerate(corrected_corpus):
    print(f"데이터 행 {index + 1}: {corrected_sentences}")


분리된 문장: ['지금 너 스스로를 죽여달라고 애원하는 것인가?', ' 아닙니다. 죄송합니다.', ' 죽을 거면 혼자 죽지 우리까지 사건에 휘말리게 해? 진짜 죽여버리고 싶게.', ' 정말 잘못했습니다.', ' 너가 선택해. 너가 죽을래 네 가족을 죽여줄까.', ' 죄송합니다. 정말 잘못했습니다.', ' 너에게는 선택권이 없어. 선택 못한다면 너와 네 가족까지 모조리 죽여버릴거야.', ' 선택 못하겠습니다. 한번만 도와주세요.', ' 그냥 다 죽여버려야겠군. 이의 없지?', ' 제발 도와주세요.']
분리된 문장: ['길동경찰서입니다.', '9시 40분 마트에 폭발물을 설치할거다.', '네?', '똑바로 들어 한번만 더 얘기한다.', '장난전화 걸지 마시죠. ', '9시 40분 마트에 폭발물이 터지면 다 죽는거야. ', '장난전화는 업무방해죄에 해당됩니다.', '판단은 너에게 달려있다. 길동경찰서에도 폭발물 터지면 꽤나 재미있겠지.', '선생님 진정하세요.', '난 이야기했어. 경고했다는 말이야.']
분리된 문장: ['너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.', '그만해. 니들 놀리는거 재미없어.', '지영아 너가 키 160이지? 그럼 재는 160도 안돼는거네?', '너 군대도 안가고 좋겠다.', '니들이 나 작은데 보태준거 있냐?', '난쟁이들도 장가가고하던데. 너도 희망을 가져봐 ', '더이상 하지마라. ', '그 키크는 수술도 있대잖아? 니네 엄마는 그거 안해주디?', '나람 해줬어. 저 키로 어찌살아.', '제발 그만 괴롭히라고!']
데이터 행 1: ['지금 너 스스로를 죽여달라고 애원하는 것인가?', '아닙니다. 죄송합니다.', '죽을 거면 혼자 죽 지 우리까지 사건에 휘말리게 해? 진짜 죽 여버리고 싶게.', '정말 잘못했습니다.', '너가 선택해. 너가 죽을래 네 가족을 죽여줄까.', '죄송합니다. 정말 잘못했습니다.', '너에게는 선택권이 없어. 선택 못한다면 너와 네 가족까지 모조리 죽 여버릴 거야.', '선택 못하겠습니다. 한 번만

### 맞춤법 교정 - kospell

In [ ]:
# !go get -u github.com/ultimate-go-korean/kospell

# GitHub에서 KoSpell 클론
!git clone https://github.com/ultimate-go-korean/kospell.git

# KoSpell 디렉토리로 이동하여 설치
%cd kospell
!python setup.py install


Cloning into 'kospell'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 32 (delta 9), reused 26 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (32/32), 14.42 KiB | 1.31 MiB/s, done.
Resolving deltas: 100% (9/9), done.
/content/kospell
python3: can't open file '/content/kospell/setup.py': [Errno 2] No such file or directory


In [ ]:
# 경로 추가 (필요할 경우)
import sys
sys.path.append('/content/kospell')

# KoSpell import
from kospell import spell_checker


ImportError: cannot import name 'spell_checker' from 'kospell' (unknown location)

In [ ]:
from kospell import spell_checker

# 맞춤법 교정할 문장
text = "이 문장에는 맞춤법 오류가 있읍니다."

# 맞춤법 교정 수행
result = spell_checker.check(text)

# 결과 출력
print("원문:", text)
print("교정된 문장:", result.as_dict()['checked'])


ImportError: cannot import name 'spell_checker' from 'kospell' (unknown location)

### Keras Tokenizer (aiffel node)

In [ ]:
# keras Tokenizer

def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=1000,
        filters=' ',
        oov_token="<unk>")
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=20)

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  1 210   1 ...   1  41   1]
 [  1   1   1 ...   1   1 396]
 [  1  96   1 ...   1 125   1]
 ...
 [485   3 113 ...  27   1   1]
 [  1   1 152 ... 350   1   1]
 [  1   1   1 ... 156 114   1]] <keras.src.legacy.preprocessing.text.Tokenizer object at 0x7a541ba0e7d0>


In [ ]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : 내가
3 : 너
4 : 좀
5 : 다
6 : 왜
7 : 나
8 : 진짜
9 : 야
10 : 지금


In [ ]:
X_train = tensor[:3500]
y_train = train_data['class'][:3500]
X_val = tensor[3500:3850]
y_val = train_data['class'][3500:3850]
X_test = tensor[3850:]
y_test = train_data['class'][3850:]

In [ ]:
sweep_config = {
    "name": "sweep_test_nlp",
    "metric": {"name": "val_loss", "goal": "minimize"},
    "method": "random",
    "parameters": {
        "learning_rate" : {
            "min" : 0.001,
            "max" : 0.1
            },
        "epoch" : {
            "distribution" : "int_uniform",
            "min" : 5,
            "max" : 10
            }

        }
    }